# Сбор текстов с сайта Банки.ру

    Для реализации большинства алгоритмов анализа тональности необходим обучающий массив данных (набор отзывов, для которых известна эмоциональная окраска). Так как на сегодняшний день существует очень мало публичных русскоязычных коллекций отзывов, которые можно было бы использовать для решения задачи классификации текстов, и не обнаружено ни одной русскоязычной публичной коллекции отзывов о банках, было принято решение подготовить собственный корпус текстов.
    
    На сайте Банки.ру оценки пользователей напрямую связаны с тональностью текста: оценка «5» соответствует только отзывам, содержащим описание положительных характеристик банка, продукта или услуги или выражение благодарности сотрудникам банка, поэтому данный веб-сайт был выбран для составления корпуса.
    
 

### Импорт библиотек

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

import nest_asyncio
import aiohttp
import asyncio

### Сбор корпуса

    Изучив структуру сайта Банки.ру, обнаружили, что полный текст каждого отдельного отзыва находится в соответствующем ему HTML-файле, поэтому сначала необходимо собрать все ссылки на необходимые файлы. 

In [2]:
def links(page_number, rate):
    url = f'https://www.banki.ru/services/responses/list/?page={page_number}&is_countable=on&rate={rate}'
    response = requests.get(url, headers={'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'})
    
    if response.status_code != 200:
        print(f"Ошибка при запросе страницы {page_number}: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    h3_elements = soup.find_all('h3', class_='text-weight-medium text-size-3 ldecc766d')
    
    for h3 in h3_elements:
        a_element = h3.find('a', class_='link-simple')
        if a_element:
            link = a_element['href']
            full_link = f"https://www.banki.ru{link}" 
            links.append(full_link)
    
    return links


In [3]:
pos_data = []

for page_number in range(1, 200):
    print(f"Обрабатываем страницу {page_number}...")
    urls = links(page_number=page_number, rate=5)  
    if not urls:  
        break
    pos_data.extend(urls)  

print(f"Собрано ссылок: {len(pos_data)}")

Обрабатываем страницу 1...
Обрабатываем страницу 2...
Обрабатываем страницу 3...
Обрабатываем страницу 4...
Обрабатываем страницу 5...
Обрабатываем страницу 6...
Обрабатываем страницу 7...
Обрабатываем страницу 8...
Обрабатываем страницу 9...
Обрабатываем страницу 10...
Обрабатываем страницу 11...
Обрабатываем страницу 12...
Обрабатываем страницу 13...
Обрабатываем страницу 14...
Обрабатываем страницу 15...
Обрабатываем страницу 16...
Обрабатываем страницу 17...
Обрабатываем страницу 18...
Обрабатываем страницу 19...
Обрабатываем страницу 20...
Обрабатываем страницу 21...
Обрабатываем страницу 22...
Обрабатываем страницу 23...
Обрабатываем страницу 24...
Обрабатываем страницу 25...
Обрабатываем страницу 26...
Обрабатываем страницу 27...
Обрабатываем страницу 28...
Обрабатываем страницу 29...
Обрабатываем страницу 30...
Обрабатываем страницу 31...
Обрабатываем страницу 32...
Обрабатываем страницу 33...
Обрабатываем страницу 34...
Обрабатываем страницу 35...
Обрабатываем страницу 36...
О

In [4]:
neg_data = []

for page_number in range(1, 200):
    print(f"Обрабатываем страницу {page_number}...")
    urls = links(page_number=page_number, rate=1)  
    if not urls:  
        break
    neg_data.extend(urls)  

print(f"Собрано ссылок: {len(pos_data)}")

Обрабатываем страницу 1...
Обрабатываем страницу 2...
Обрабатываем страницу 3...
Обрабатываем страницу 4...
Обрабатываем страницу 5...
Обрабатываем страницу 6...
Обрабатываем страницу 7...
Обрабатываем страницу 8...
Обрабатываем страницу 9...
Обрабатываем страницу 10...
Обрабатываем страницу 11...
Обрабатываем страницу 12...
Обрабатываем страницу 13...
Обрабатываем страницу 14...
Обрабатываем страницу 15...
Обрабатываем страницу 16...
Обрабатываем страницу 17...
Обрабатываем страницу 18...
Обрабатываем страницу 19...
Обрабатываем страницу 20...
Обрабатываем страницу 21...
Обрабатываем страницу 22...
Обрабатываем страницу 23...
Обрабатываем страницу 24...
Обрабатываем страницу 25...
Обрабатываем страницу 26...
Обрабатываем страницу 27...
Обрабатываем страницу 28...
Обрабатываем страницу 29...
Обрабатываем страницу 30...
Обрабатываем страницу 31...
Обрабатываем страницу 32...
Обрабатываем страницу 33...
Обрабатываем страницу 34...
Обрабатываем страницу 35...
Обрабатываем страницу 36...
О

In [5]:
pos_data[:10]

['https://www.banki.ru/services/responses/bank/response/11747735/',
 'https://www.banki.ru/services/responses/bank/response/11746786/',
 'https://www.banki.ru/services/responses/bank/response/11745617/',
 'https://www.banki.ru/services/responses/bank/response/11745592/',
 'https://www.banki.ru/services/responses/bank/response/11745555/',
 'https://www.banki.ru/services/responses/bank/response/11745506/',
 'https://www.banki.ru/services/responses/bank/response/11743852/',
 'https://www.banki.ru/services/responses/bank/response/11743442/',
 'https://www.banki.ru/services/responses/bank/response/11743377/',
 'https://www.banki.ru/services/responses/bank/response/11743050/']

In [6]:
neg_data[:10]

['https://www.banki.ru/services/responses/bank/response/11748640/',
 'https://www.banki.ru/services/responses/bank/response/11747156/',
 'https://www.banki.ru/services/responses/bank/response/11745363/',
 'https://www.banki.ru/services/responses/bank/response/11744968/',
 'https://www.banki.ru/services/responses/bank/response/11744538/',
 'https://www.banki.ru/services/responses/bank/response/11744500/',
 'https://www.banki.ru/services/responses/bank/response/11744160/',
 'https://www.banki.ru/services/responses/bank/response/11743394/',
 'https://www.banki.ru/services/responses/bank/response/11743198/',
 'https://www.banki.ru/services/responses/bank/response/11743172/']

    Далее, тем же методом с кода страниц, находящихся по каждой из полученных ссылок, сохраняем сам текст отзыва  и соответствующие ему метаданные, а именно: дату написания отзыва, тему обращения, название банка и город.

In [7]:
nest_asyncio.apply()  

async def fetch(session, url):
    async with session.get(url, headers={'User-agent': 'Mozilla/5.0'}) as response:
        return await response.text()

async def bank_data(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        pages = await asyncio.gather(*tasks)
        a = 0

        results = []
        for a, page in enumerate(pages):
            if page:
                print(f"Обрабатываем ссылку {a + 1} / 4975...")
                soup = BeautifulSoup(page, 'html.parser')

                text = [div.text for div in soup.find_all('div', class_='lb1789875 markdown-inside markdown-inside--list-type_circle-fill')]
                date = soup.find_all('span', class_='l10fac986')[0].text if soup.find_all('span', class_='l10fac986') else None
                city = [span.text for span in soup.find_all('span', class_='l3a372298')]
                bank_name = [div.text for div in soup.find_all('div', class_='l5b3cd260')]
                theme = soup.find_all('h1', class_='text-header-0 le856f50c')[0].text if soup.find_all('h1', class_='text-header-0 le856f50c') else None

                results.append({
                    'url': urls[a],
                    'text': text,
                    'date': date,
                    'city': city,
                    'bank_name': bank_name,
                    'theme': theme
                })

        return results


urls = [i for i in pos_data]
results_pos = await bank_data(urls)

Обрабатываем ссылку 1 / 4975...
Обрабатываем ссылку 2 / 4975...
Обрабатываем ссылку 3 / 4975...
Обрабатываем ссылку 4 / 4975...
Обрабатываем ссылку 5 / 4975...
Обрабатываем ссылку 6 / 4975...
Обрабатываем ссылку 7 / 4975...
Обрабатываем ссылку 8 / 4975...
Обрабатываем ссылку 9 / 4975...
Обрабатываем ссылку 10 / 4975...
Обрабатываем ссылку 11 / 4975...
Обрабатываем ссылку 12 / 4975...
Обрабатываем ссылку 13 / 4975...
Обрабатываем ссылку 14 / 4975...
Обрабатываем ссылку 15 / 4975...
Обрабатываем ссылку 16 / 4975...
Обрабатываем ссылку 17 / 4975...
Обрабатываем ссылку 18 / 4975...
Обрабатываем ссылку 19 / 4975...
Обрабатываем ссылку 20 / 4975...
Обрабатываем ссылку 21 / 4975...
Обрабатываем ссылку 22 / 4975...
Обрабатываем ссылку 23 / 4975...
Обрабатываем ссылку 24 / 4975...
Обрабатываем ссылку 25 / 4975...
Обрабатываем ссылку 26 / 4975...
Обрабатываем ссылку 27 / 4975...
Обрабатываем ссылку 28 / 4975...
Обрабатываем ссылку 29 / 4975...
Обрабатываем ссылку 30 / 4975...
Обрабатываем ссылку

In [8]:
urls = [i for i in neg_data]
results_neg = await bank_data(urls)

Обрабатываем ссылку 1 / 4975...
Обрабатываем ссылку 2 / 4975...
Обрабатываем ссылку 3 / 4975...
Обрабатываем ссылку 4 / 4975...
Обрабатываем ссылку 5 / 4975...
Обрабатываем ссылку 6 / 4975...
Обрабатываем ссылку 7 / 4975...
Обрабатываем ссылку 8 / 4975...
Обрабатываем ссылку 9 / 4975...
Обрабатываем ссылку 10 / 4975...
Обрабатываем ссылку 11 / 4975...
Обрабатываем ссылку 12 / 4975...
Обрабатываем ссылку 13 / 4975...
Обрабатываем ссылку 14 / 4975...
Обрабатываем ссылку 15 / 4975...
Обрабатываем ссылку 16 / 4975...
Обрабатываем ссылку 17 / 4975...
Обрабатываем ссылку 18 / 4975...
Обрабатываем ссылку 19 / 4975...
Обрабатываем ссылку 20 / 4975...
Обрабатываем ссылку 21 / 4975...
Обрабатываем ссылку 22 / 4975...
Обрабатываем ссылку 23 / 4975...
Обрабатываем ссылку 24 / 4975...
Обрабатываем ссылку 25 / 4975...
Обрабатываем ссылку 26 / 4975...
Обрабатываем ссылку 27 / 4975...
Обрабатываем ссылку 28 / 4975...
Обрабатываем ссылку 29 / 4975...
Обрабатываем ссылку 30 / 4975...
Обрабатываем ссылку

In [9]:
results_pos[:3]

[{'url': 'https://www.banki.ru/services/responses/bank/response/11747735/',
  'text': ['\n\t30 июня был открыт накопительный счёт, процент за июнь не выплачен, ставка не отобразилась. Четкой прописанной ставки около счета в мобильном приложении нет. Пришлось узнавать через чат. Проценты за июль выплачены в размере 17 годовых. В августе    также четко процент накопительного счета не отобразился. В таблице процентов банка 17 процентов поменялись на 19. \r\n\r\nЛично я получила за август  вознаграждение по ставке 5 процентов, т.к. якобы это третий месяц вклада, приветственная ставка не применяется. Вопрос: почему четко не отображен процент счета в личном кабинете? Деньги можно было вывести под более удачный процент.Потерянная выгода в банке МТС. Последний день месяца засчитывается в приветственную ставку. Информации по счету достоверной нет, расплывчатая таблица процентов.\n'],
  'date': '\n\t\t\t22.09.2024 17:28\n\t\t',
  'city': ['Нижний Тагил (Свердловская область)'],
  'bank_name': ['

In [10]:
results_neg[:3]

[{'url': 'https://www.banki.ru/services/responses/bank/response/11748640/',
  'text': ['\nПосле оплаты не пришло на почту письмо с ваучером. До техподдержки дозвониться не смог. Первый раз попробовал позвонить в субботу, но так как был выходной, не удивился, что свободных операторов не нашлось. Во второй раз сделал звонок в понедельник, сначала бот отправил ссылку на чат, которая в приложении не открылась (происходила ошибка). Позвонил во второй раз, попросил перевести на оператора, время ожидания было слишком долгим. Самостоятельно чат в приложении банка также не нашёл (в меню есть только пункт "Позвонить нам"). На сайте чат поддержки не работает, кнопка отправки не кликабельна.\n'],
  'date': '\n\t\t\t23.09.2024 14:47\n\t\t',
  'city': ['Калининград (Калининградская область)'],
  'bank_name': ['МТС Банк', 'user-247419471967'],
  'theme': '\n\t\tПроблема с оплатой chatGPT\n\t'},
 {'url': 'https://www.banki.ru/services/responses/bank/response/11747156/',
  'text': ['\nпрошла неделя, ка

In [11]:
df_pos = pd.DataFrame(results_pos)

In [12]:
df_neg = pd.DataFrame(results_neg)

    Собранные данные очищаем с помощью регулярных выражений и на основе них формируем датафрейм.

In [ ]:
import re  # Импортируем библиотеку регулярных выражений

# Определяем функцию для очистки текстового столбца
def clean_text_column(df, column_name):
    cleaned_text = []  # Создаем список для хранения очищенных текстов
    
    # Проходим по всем значениям в указанном столбце DataFrame
    for value in df[column_name]:  
        # Находим и объединяем все фрагменты текста, начинающиеся с буквы (русские) 
        string = ' '.join(re.findall(r'[А-Яа-я].+', str(value)))
        
        # Убираем лишние символы и форматы, оставляя только допустимые
        string = re.sub(r'\\r|\\n|[^А-Яа-я0-9\s.,\-–"]+', '', string)
        # Удаляем HTML-теги </p> из текста
        string = re.sub('</p>', '', string)
        
        # Добавляем очищенное значение в список
        cleaned_text.append(string)
    
    # Возвращаем список очищенных текстов
    return cleaned_text

# Применяем очистку текста к столбцу 'text' в DataFrame df_neg
df_neg['text'] = clean_text_column(df_neg, 'text')
# Применяем очистку текста к столбцу 'text' в DataFrame df_pos
df_pos['text'] = clean_text_column(df_pos, 'text')

In [13]:
def clean_text_column(df, column_name):
    cleaned_text = []    
    for value in df[column_name]:  
        string = ' '.join(re.findall(r'[А-Яа-я].+', str(value)))
        string = re.sub(r'\\r|\\n|[^А-Яа-я0-9\s.,\-–"]+', '', string)
        string = re.sub('</p>', '', string)
        
        cleaned_text.append(string)
    
    return cleaned_text

df_neg['text'] = clean_text_column(df_neg, 'text')
df_pos['text'] = clean_text_column(df_pos, 'text')


In [14]:
def clean_and_convert_dates(df, column_name):
    cleaned_dates = []
    
    for value in df[column_name]:  
        string = re.sub(r'\n\t\t\t|\n\t\t', '', str(value))
        
        try:
            date = pd.to_datetime(string, errors='coerce')  
        except Exception as e:
            date = pd.NaT  
            
        cleaned_dates.append(date)
    
    return cleaned_dates


df_neg['date'] = clean_and_convert_dates(df_neg, 'date')
df_pos['date'] = clean_and_convert_dates(df_pos, 'date')

C:\Users\js\AppData\Local\Temp\ipykernel_8996\54785669.py:8: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  date = pd.to_datetime(string, errors='coerce')


In [15]:
def clean_city_names(df, column_name):
    cleaned_cities = []

    for value in df[column_name]:
        string = re.sub(r"[\[\]\'\"]", '', str(value))
        cleaned_cities.append(string.strip())  # Удаляем лишние пробелы по краям

    return cleaned_cities

df_neg['city'] = clean_city_names(df_neg, 'city')
df_pos['city'] = clean_city_names(df_pos, 'city')



In [16]:
def extract_bank_names(row):
    value = row['bank_name']
    bank_names = []

    
    match = re.search(r'\[(.*?)\]', str(value))
    if match:
        bank_names_in_brackets = match.group(1)
        bank_parts = [name.strip() for name in bank_names_in_brackets.split(',')]

        
        if "Администратор" in bank_parts[0]:
            bank_names.append(bank_parts[1] if len(bank_parts) > 1 else None)
        else:
            bank_names.append(bank_parts[0])
    else:
        bank_names.append(None)  

    return bank_names[0]  


df_neg['bank_name'] = df_neg.apply(extract_bank_names, axis=1)
df_pos['bank_name'] = df_pos.apply(extract_bank_names, axis=1)


In [17]:
def clean_theme(df, column_name):
    cleaned_themes = []

    for value in df[column_name]:
        string = re.sub(r'\n\t\t|\n\t', '', str(value))
        cleaned_themes.append(string.strip()) 

    return cleaned_themes

df_neg['theme'] = clean_theme(df_neg, 'theme')
df_pos['theme'] = clean_theme(df_pos, 'theme')


In [18]:
df_neg.head(5)


,url,text,date,city,bank_name,theme
0,https://www.banki.ru/services/responses/bank/r...,После оплаты не пришло на почту письмо с вауче...,2024-09-23 14:47:00,Калининград (Калининградская область),'МТС Банк',Проблема с оплатой chatGPT
1,https://www.banki.ru/services/responses/bank/r...,"прошла неделя, как курьер привз мне дебетовую ...",2024-09-22 04:10:00,Красноярск (Красноярский край),'Уралсиб',Активация карты
2,https://www.banki.ru/services/responses/bank/r...,В сервисе МТС деньги при переводе 150 рублей с...,2024-09-20 13:08:00,Усть-Лабинск (Краснодарский край),'МТС Банк',Нужна проверка. Заметили подозрительную активн...
3,https://www.banki.ru/services/responses/bank/r...,Пользоваться услугами банка МТС решать вам. Я ...,2024-09-20 09:09:00,Буденновск (Ставропольский край),'МТС Банк',Моему возмущению нет предела! Часть первая
4,https://www.banki.ru/services/responses/bank/r...,"Здравствуйте Являюсь, к сожалению, клиентом ба...",2024-09-19 20:42:00,Ульяновск (Ульяновская область),'МТС Банк',МТС банк - самый ужасный банк в моей жизни. Не...


In [20]:
def remove_quotes(df, column_name):
    df[column_name] = df[column_name].str.replace('"', '', regex=False).str.replace("'", '', regex=False)

df_neg['bank_name'] = remove_quotes(df_neg, 'bank_name')
df_pos['bank_name'] = remove_quotes(df_pos, 'bank_name')
df_neg.head(5)

,url,text,date,city,bank_name,theme
0,https://www.banki.ru/services/responses/bank/r...,После оплаты не пришло на почту письмо с вауче...,2024-09-23 14:47:00,Калининград (Калининградская область),None,Проблема с оплатой chatGPT
1,https://www.banki.ru/services/responses/bank/r...,"прошла неделя, как курьер привз мне дебетовую ...",2024-09-22 04:10:00,Красноярск (Красноярский край),None,Активация карты
2,https://www.banki.ru/services/responses/bank/r...,В сервисе МТС деньги при переводе 150 рублей с...,2024-09-20 13:08:00,Усть-Лабинск (Краснодарский край),None,Нужна проверка. Заметили подозрительную активн...
3,https://www.banki.ru/services/responses/bank/r...,Пользоваться услугами банка МТС решать вам. Я ...,2024-09-20 09:09:00,Буденновск (Ставропольский край),None,Моему возмущению нет предела! Часть первая
4,https://www.banki.ru/services/responses/bank/r...,"Здравствуйте Являюсь, к сожалению, клиентом ба...",2024-09-19 20:42:00,Ульяновск (Ульяновская область),None,МТС банк - самый ужасный банк в моей жизни. Не...


In [21]:
df_pos['coloring'] = 'pos'
df_neg['coloring'] = 'neg'

combined_df = pd.concat([df_pos, df_neg], ignore_index=True)

combined_df = combined_df.sample(frac=1, random_state=1).reset_index(drop=True)



In [22]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9903 entries, 0 to 9902
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   url        9903 non-null   object        
 1   text       9903 non-null   object        
 2   date       9903 non-null   datetime64[ns]
 3   city       9903 non-null   object        
 4   bank_name  0 non-null      object        
 5   theme      9903 non-null   object        
 6   coloring   9903 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 541.7+ KB


In [23]:
combined_df = combined_df.drop_duplicates()

In [24]:
combined_df.to_csv('data.csv') 